In [1]:
# Import the necessary libraries
%matplotlib inline
import numpy as np
import pandas as pd

from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import KFold
import pickle

from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score as acs_score

from src.data.data_fetcher import get_raw_data
from src.features.preprocess_data import get_preprocessed_test_data, fetch_preprocessed_data
pd.set_option('display.max_columns', 200)

from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings('ignore')
%pip install catboost

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\gunna\Documents\Maskinlæring\Prosjekt\power-predictor\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
# Prepare data
train_a, train_b, train_c, X_train_estimated_a, X_train_estimated_b, X_train_estimated_c, X_train_observed_a, X_train_observed_b, X_train_observed_c, X_test_estimated_a, X_test_estimated_b, X_test_estimated_c = get_raw_data()

X_train_obs_combined, X_val_obs_combined, y_train_obs_combined, y_val_obs_combined, X_train_est_combined, X_val_est_combined, y_train_est_combined, y_val_est_combined = fetch_preprocessed_data()
x_test_whole = get_preprocessed_test_data()

x_whole = pd.concat([X_train_obs_combined, X_val_obs_combined])
y_whole = pd.concat([y_train_obs_combined, y_val_obs_combined])
x_whole.reset_index(drop=True, inplace=True)
y_whole.reset_index(drop=True, inplace=True)

x_whole_obs = pd.concat([X_train_obs_combined, X_val_obs_combined])
y_whole_obs = pd.concat([y_train_obs_combined, y_val_obs_combined])

x_whole_est = pd.concat([X_train_est_combined, X_val_est_combined])
y_whole_est = pd.concat([y_train_est_combined, y_val_est_combined])

x_whole.head()

X_test_estimated_a.shape = (2880, 47), X_test_estimated_b.shape = (2880, 47), X_test_estimated_c.shape = (2880, 47)
After temporal alignment
X_test_estimated_a.shape = (720, 47), X_test_estimated_b.shape = (720, 47), X_test_estimated_c.shape = (720, 47)
X_test_estimated_a_processed.shape = (720, 46), X_test_estimated_b_processed.shape = (720, 46), X_test_estimated_c_processed.shape = (720, 46)


,absolute_humidity_2m:gm3,air_density_2m:kgm3,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,is_day:idx,is_in_shadow:idx,precip_5min:mm,precip_type_5min:idx,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sun_azimuth:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,location_a,location_b,location_c,sin_day_of_year,cos_day_of_year,sin_hour,cos_hour,sun_product,modified_solar_elevation,effective_radiation,time_since_prediction,cloud_ratio,cloud_cover_over_30%,sun_addition,is_freezing,is_snow,is_rain
0,15.2,1.18175,1.351672e+06,440.600006,7231.174805,0.0,291.250000,95.500000,294605.53125,271.225006,872357.56250,54.224998,1.0,0.0,0.0,0.0,1007.500000,0.0,0.0,72.375000,92.172501,0.00,294.625000,98.925003,48152.375000,2.125,-2.000,-0.750,0.0,1,0,0,0.027950,-0.999609,1.000000,6.123234e-17,25901.988281,0.455669,0.645392,0,0.548142,1,366.725006,0,0,0
1,13.7,1.19250,2.456032e+06,637.599976,1410.500000,0.0,289.399994,145.525009,447652.68750,5.800000,11749.87500,99.675003,1.0,0.0,0.0,0.0,1001.400024,0.0,0.1,80.699997,236.257751,0.10,292.100006,100.000000,5903.875000,1.350,-1.225,0.525,0.0,0,1,0,-0.310857,-0.950457,-0.500000,-8.660254e-01,844.045105,0.611489,0.004784,0,0.996750,1,151.325012,0,0,0
2,10.5,1.20150,2.172293e+05,107.400002,2315.000000,0.0,285.299988,42.200001,100314.65625,65.099998,135992.90625,23.299999,1.0,0.0,0.0,0.0,1006.849976,0.0,0.0,59.049999,85.390503,0.00,291.700012,43.549999,49974.699219,3.375,-0.400,3.350,0.0,1,0,0,-0.804826,-0.593511,0.965926,2.588190e-01,2747.219971,0.164161,0.626034,0,0.535017,0,107.300003,0,0,0
3,6.3,1.24800,0.000000e+00,0.000000,131.375000,0.0,277.100006,0.000000,0.00000,0.000000,0.00000,99.574997,0.0,1.0,0.0,0.0,992.949951,0.0,0.0,93.650002,127.701752,0.35,278.100006,99.974998,3399.375000,6.000,5.950,-0.500,0.0,1,0,0,-0.141444,0.989946,0.965926,-2.588190e-01,0.000000,0.000000,0.000000,0,0.995999,1,0.000000,0,0,0
4,6.3,1.27175,0.000000e+00,0.000000,8580.474609,0.0,277.000000,0.000000,0.00000,0.000000,0.00000,49.400002,0.0,1.0,0.0,0.0,1012.200012,0.0,0.0,81.649994,346.724243,0.00,278.600006,98.824997,27873.150391,1.450,1.450,-0.275,0.0,1,0,0,0.931210,-0.364483,-0.500000,8.660254e-01,0.000000,0.000000,0.000000,0,0.499874,1,0.000000,0,0,0


In [3]:
x_whole["pv_measurement"] = y_whole
df_shuffled = x_whole.sample(frac=1, random_state=42).reset_index(drop=True)


In [4]:
x_whole_a = df_shuffled[df_shuffled['location_a'] == 1]
x_whole_b = df_shuffled[df_shuffled['location_b'] == 1]
x_whole_c = df_shuffled[df_shuffled['location_c'] == 1]

y_whole_a = x_whole_a["pv_measurement"]
x_whole_a = x_whole_a.drop("pv_measurement", axis = 1)
x_whole_a = x_whole_a.drop('location_a', axis = 1)
x_whole_a = x_whole_a.drop('location_b', axis = 1)
x_whole_a = x_whole_a.drop('location_c', axis = 1)

y_whole_b = x_whole_b["pv_measurement"]
x_whole_b = x_whole_b.drop("pv_measurement", axis = 1)
x_whole_b = x_whole_b.drop('location_a', axis = 1)
x_whole_b = x_whole_b.drop('location_b', axis = 1)
x_whole_b = x_whole_b.drop('location_c', axis = 1)

y_whole_c = x_whole_c["pv_measurement"]
x_whole_c = x_whole_c.drop("pv_measurement", axis = 1)
x_whole_c = x_whole_c.drop('location_a', axis = 1)
x_whole_c = x_whole_c.drop('location_b', axis = 1)
x_whole_c = x_whole_c.drop('location_c', axis = 1)

In [5]:
cat_features = ["time_since_prediction"]
reg_a = CatBoostRegressor(
    loss_function='MAE',  # Mean Absolute Error
    verbose=10,  # Output training progress every 200 iterations
    max_depth=9,
    cat_features=cat_features,
)
reg_a.fit(x_whole_a, y_whole_a)

0:	learn: 658.0950153	total: 176ms	remaining: 2m 56s
10:	learn: 517.3819133	total: 531ms	remaining: 47.7s
20:	learn: 413.9635077	total: 890ms	remaining: 41.5s
30:	learn: 346.5512327	total: 1.22s	remaining: 38.1s
40:	learn: 299.8139158	total: 1.59s	remaining: 37.2s
50:	learn: 268.5239696	total: 1.94s	remaining: 36.1s
60:	learn: 247.5511176	total: 2.28s	remaining: 35.2s
70:	learn: 234.0310765	total: 2.61s	remaining: 34.1s
80:	learn: 223.7893084	total: 2.95s	remaining: 33.5s
90:	learn: 216.7406024	total: 3.29s	remaining: 32.8s
100:	learn: 211.8823206	total: 3.65s	remaining: 32.5s
110:	learn: 208.5067326	total: 3.96s	remaining: 31.7s
120:	learn: 205.2386774	total: 4.29s	remaining: 31.1s
130:	learn: 203.6602636	total: 4.64s	remaining: 30.8s
140:	learn: 201.7429775	total: 4.96s	remaining: 30.2s
150:	learn: 200.4176615	total: 5.29s	remaining: 29.8s
160:	learn: 198.9534203	total: 5.62s	remaining: 29.3s
170:	learn: 198.0047911	total: 5.97s	remaining: 28.9s
180:	learn: 197.3171631	total: 6.3s	re

In [6]:
reg_b = CatBoostRegressor(
    loss_function='MAE',  # Mean Absolute Error
    verbose=10,  # Output training progress every 200 iterations
    max_depth=9,
    cat_features=cat_features,
)
reg_b.fit(x_whole_b, y_whole_b)

0:	learn: 100.5712039	total: 34.9ms	remaining: 34.9s
10:	learn: 79.0003750	total: 342ms	remaining: 30.7s
20:	learn: 62.6977077	total: 654ms	remaining: 30.5s
30:	learn: 51.0944114	total: 967ms	remaining: 30.2s
40:	learn: 42.8433594	total: 1.27s	remaining: 29.6s
50:	learn: 37.2333161	total: 1.56s	remaining: 29s
60:	learn: 33.4431779	total: 1.9s	remaining: 29.2s
70:	learn: 30.8580179	total: 2.23s	remaining: 29.2s
80:	learn: 29.0425222	total: 2.53s	remaining: 28.7s
90:	learn: 27.7034781	total: 2.83s	remaining: 28.2s
100:	learn: 26.6796253	total: 3.14s	remaining: 28s
110:	learn: 25.9611489	total: 3.46s	remaining: 27.7s
120:	learn: 25.3384504	total: 3.77s	remaining: 27.4s
130:	learn: 24.8572425	total: 4.07s	remaining: 27s
140:	learn: 24.4783436	total: 4.37s	remaining: 26.6s
150:	learn: 24.1552081	total: 4.67s	remaining: 26.3s
160:	learn: 23.8876762	total: 4.95s	remaining: 25.8s
170:	learn: 23.5623209	total: 5.24s	remaining: 25.4s
180:	learn: 23.3456179	total: 5.55s	remaining: 25.1s
190:	lear

In [7]:
reg_c = CatBoostRegressor(
    loss_function='MAE',  # Mean Absolute Error
    verbose=10,  # Output training progress every 200 iterations
    max_depth=9,
    cat_features=cat_features,
)
reg_c.fit(x_whole_c, y_whole_c)

0:	learn: 83.2334597	total: 31.5ms	remaining: 31.5s
10:	learn: 65.9203474	total: 352ms	remaining: 31.7s
20:	learn: 53.1994602	total: 655ms	remaining: 30.6s
30:	learn: 43.2343675	total: 959ms	remaining: 30s
40:	learn: 36.1867599	total: 1.25s	remaining: 29.2s
50:	learn: 31.4104314	total: 1.56s	remaining: 29s
60:	learn: 27.9962662	total: 1.86s	remaining: 28.6s
70:	learn: 25.5782295	total: 2.16s	remaining: 28.3s
80:	learn: 23.8573667	total: 2.44s	remaining: 27.7s
90:	learn: 22.6209649	total: 2.71s	remaining: 27.1s
100:	learn: 21.7226433	total: 3.01s	remaining: 26.8s
110:	learn: 21.0584049	total: 3.3s	remaining: 26.5s
120:	learn: 20.5738404	total: 3.59s	remaining: 26.1s
130:	learn: 20.1750451	total: 3.88s	remaining: 25.8s
140:	learn: 19.8605535	total: 4.18s	remaining: 25.4s
150:	learn: 19.6105540	total: 4.46s	remaining: 25.1s
160:	learn: 19.3410620	total: 4.76s	remaining: 24.8s
170:	learn: 19.1011894	total: 5.06s	remaining: 24.5s
180:	learn: 18.8958487	total: 5.35s	remaining: 24.2s
190:	lea

In [8]:
x_whole_a = x_test_whole[x_test_whole['location_a'] == 1]
x_whole_b = x_test_whole[x_test_whole['location_b'] == 1]
x_whole_c = x_test_whole[x_test_whole['location_c'] == 1]

x_whole_a = x_whole_a.drop('location_a', axis = 1)
x_whole_a = x_whole_a.drop('location_b', axis = 1)
x_whole_a = x_whole_a.drop('location_c', axis = 1)

x_whole_b = x_whole_b.drop('location_a', axis = 1)
x_whole_b = x_whole_b.drop('location_b', axis = 1)
x_whole_b = x_whole_b.drop('location_c', axis = 1)

x_whole_c = x_whole_c.drop('location_a', axis = 1)
x_whole_c = x_whole_c.drop('location_b', axis = 1)
x_whole_c = x_whole_c.drop('location_c', axis = 1)

In [9]:
x_whole_a.head()

,absolute_humidity_2m:gm3,air_density_2m:kgm3,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,is_day:idx,is_in_shadow:idx,precip_5min:mm,precip_type_5min:idx,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sun_azimuth:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,sin_day_of_year,cos_day_of_year,sin_hour,cos_hour,sun_product,modified_solar_elevation,effective_radiation,time_since_prediction,cloud_ratio,cloud_cover_over_30%,sun_addition,is_freezing,is_snow,is_rain
0,4.325,1.28675,0.000000,0.000,1061.550049,0.0,271.650024,0.000000,0.000000,0.000000,0.000000,74.949997,0.00,1.0,0.0,0.0,1006.799988,0.0,0.0,80.275002,16.026501,0.0,273.799988,74.949997,29907.500000,3.950,2.100,3.35,0.0,0.880683,-0.473706,0.000000,1.000000,0.000000,0.000000,0.000000,1,1.0,1,0.000000,0,0,0
1,4.275,1.28600,0.000000,0.000,1075.100098,0.0,271.450012,0.000000,0.000000,0.000000,0.000000,77.474998,0.00,1.0,0.0,0.0,1006.299988,0.0,0.0,79.824997,30.497250,0.0,273.799988,77.474998,29519.074219,3.825,1.925,3.30,0.0,0.880683,-0.473706,0.258819,0.965926,0.000000,0.000000,0.000000,1,1.0,1,0.000000,0,0,0
2,4.150,1.28375,0.000000,0.000,1200.400024,0.0,271.049988,0.000000,0.000000,0.000000,0.000000,88.099998,0.00,1.0,0.0,0.0,1005.799988,0.0,0.0,78.000000,44.517250,0.0,273.849976,88.099998,31009.125000,3.650,1.750,3.20,0.0,0.880683,-0.473706,0.500000,0.866025,0.000000,0.000000,0.000000,1,1.0,1,0.000000,0,0,0
3,4.025,1.28200,10124.424805,11.675,1179.849976,0.0,270.649994,9.375000,16845.226562,2.100000,3765.350098,68.599998,0.75,0.5,0.0,0.0,1005.224976,0.0,0.0,75.625000,58.083000,0.0,273.899994,68.599998,34552.500000,3.500,1.450,3.15,0.0,0.880683,-0.473706,0.707107,0.707107,19.687500,0.024650,0.371908,1,1.0,1,11.475000,0,0,0
4,3.900,1.28100,141748.593750,76.875,920.049988,0.0,270.375000,47.400002,102209.703125,25.450001,49571.199219,66.300003,1.00,0.0,0.0,0.0,1004.750000,0.0,0.0,74.224998,71.341003,0.0,273.924988,66.300003,35483.875000,3.325,1.300,3.05,0.0,0.880683,-0.473706,0.866025,0.500000,1206.330078,0.129981,0.349712,1,1.0,1,72.850006,0,0,0


In [10]:
y_pred_a = pd.DataFrame(reg_a.predict(x_whole_a))
y_pred_b = pd.DataFrame(reg_b.predict(x_whole_b))
y_pred_c = pd.DataFrame(reg_c.predict(x_whole_c))

In [11]:
y_pred = pd.concat([y_pred_a, y_pred_b, y_pred_c])
y_pred = y_pred.reset_index(drop=True)

In [12]:
# Save the model
from src.models.saving import save_predictions
from src.features.postprocess_data import postprocess_data

y_pred = postprocess_data(x_test_whole, pd.DataFrame(y_pred))
save_predictions(y_pred, 'ny catboost 3 modeller 5')

hour: 0, min_value_a: 0.0, max_value_a: 3.3, min_value_b: -0.0, max_value_b: -0.0, min_value_c: 0.0, max_value_c: 0.0
hour: 1, min_value_a: 0.0, max_value_a: 53.68, min_value_b: -0.0, max_value_b: 12.075, min_value_c: 0.0, max_value_c: 9.8
hour: 2, min_value_a: 0.0, max_value_a: 233.64000000000001, min_value_b: -0.0, max_value_b: 68.1375, min_value_c: 0.0, max_value_c: 39.2
hour: 3, min_value_a: 0.0, max_value_a: 439.12, min_value_b: -0.0, max_value_b: 138.0, min_value_c: 0.0, max_value_c: 88.2
hour: 4, min_value_a: 0.0, max_value_a: 1046.98, min_value_b: -0.0, max_value_b: 307.05, min_value_c: 0.0, max_value_c: 176.4
hour: 5, min_value_a: 0.0, max_value_a: 2049.08, min_value_b: -0.0, max_value_b: 452.8125, min_value_c: 0.0, max_value_c: 264.6
hour: 6, min_value_a: 0.0, max_value_a: 3244.78, min_value_b: -0.0, max_value_b: 681.375, min_value_c: 0.0, max_value_c: 499.8
hour: 7, min_value_a: 0.0, max_value_a: 4266.46, min_value_b: -0.0, max_value_b: 865.0875, min_value_c: 0.0, max_value_